# An Introduction to Recurrent Neural Networks

Recurrent neural networks (RNNs) have become increasingly popular in machine learning for handling sequential data. In this tutorial, we will cover [background](#1.-Background) about the architecture, a toy [training example](#2.-Training-a-toy-model), and a demo for [evaluating a larger pre-trained model](#4.-Evaluating-a-pre-trained-model).

**Date**: June 26, 2019

**Authors**: Jennifer Hu (MIT) and Ben Huh (IBM)

---

## 0. Prerequisites

To run this tutorial, you will need to install [Pytorch](https://pytorch.org/). Just follow the instructions in the docs to install the library and its dependencies. To test your installation, go ahead and run the cell below.

In [1]:
import torch

You will also need [numpy](https://www.numpy.org/), which comes automatically installed with a scientific Python distribution such as [Anaconda](https://www.anaconda.com/). Of course, you can also install it manually. Test your installation by running the cell below.

In [2]:
import numpy as np

## 1. Background: What is an RNN?

Note: The following figures were taken from excellent blog posts by [Christopher Olah](http://colah.github.io/posts/2015-08-Understanding-LSTMs/), [Jianqiang Ma](https://medium.com/@jianqiangma/all-about-recurrent-neural-networks-9e5ae2936f6e), and [Shi Yan](https://medium.com/mlreview/understanding-lstm-and-its-diagrams-37e2f46f1714).

In both machine learning and everyday life, many tasks make use of **sequential** data. Take language understanding, for example. As you comprehend each word in this sentence, you draw upon information from the previous words. RNNs have an **inductive bias** for handling this type of data, as parameters are shared across positions in the sequence.

How does this architecture compare to "normal" neural networks? Let $A$ be a neural network, $x_t$ the input, and $h_t$ the output or hidden state vector. Consider the following diagram (credit to [Christopher Olah](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)):

![RNN](img/rnn_unrolled.png)

The self-loop at $A$ represents the **recurrence** of the network. We can "unroll" it into multiple copies of the same network $A$, sending information down the temporal chain.

## 2. Training a toy RNN

**TODO @BEN**: go through character-level RNN tutorial at https://pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial.html

**TODO**: transition to explaining LSTMs

In practice, many applications use a special type of RNN called a **long short-term memory (LSTM)** network. We'll call non-LSTM RNNs "vanilla RNNs".

## 3. LSTMs: Beyond vanilla RNNs

To motivate the popularity of LSTMs, recall why we were interested in RNNs in the first place: modeling *sequences*. The idea is that items at later parts of a sequence can depend on items at earlier parts of the sequence. The distance between these related items is often called the **dependency length**.

Dependency lengths can be quite short in some cases and quite long in others, even for the same task. Returning to the language example, suppose your task is to predict the last word in a sentence. If your sentence is 

> A car has four *wheels*.

then the gap between the target word (here, "wheels") and the relevant parts of the sequence (here, "car") is relatively small. In contrast, consider the sentence 

> A car can have sentimental value for many owners for a variety of reasons, and can come in many models, sizes, and colors; nevertheless, one defining characteristic of such a machine is that it has four *wheels*.

Here, the gap between "wheels" and "car" is relatively large. Words closer to the end of the sentence such as "machine" and "four" can help guide the prediction of "wheels", but we need the word "car" to nail down the correct word.

In theory, vanilla RNNs are able to capture these dependencies of any length, but in practice, they often fail on long-term dependencies ([Bengio et al. 1994](http://ai.dinfo.unifi.it/paolo//ps/tnn-94-gradient.pdf)). LSTMs ([Hochreiter & Schmidhuber 1997](https://www.mitpressjournals.org/doi/abs/10.1162/neco.1997.9.8.1735)) turn out to be an effective solution to this problem.

### 3.1 LSTM architecture

Consider the following diagram of an LSTM (credit to [Shi Yan](https://medium.com/mlreview/understanding-lstm-and-its-diagrams-37e2f46f1714)):

![LSTM](img/lstm.png)

There's a lot going on here, but in short, LSTMs have three multiplicative gates:

1. The **input gate** controls what proportion of the input to pass to the memory cell.
2. The **forget gate** controls what proportion of the previous memory cell information to discard.
3. The **output gate** controls what proportion of the memory cell to output.


### 3.2 Further reading

[The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)

[The Deep Learning textbook](http://www.deeplearningbook.org/) (Chapter 10 is most relevant)

## 4. Evaluating a pre-trained model

Now that we've learned how to train our own small LSTM, let's examine the behavior of an LSTM that has already been pre-trained on a larger amount of data.

In particular, we will be playing around with a **language model**. A language model assigns a probability distribution over words. It is trained to perform the following task: at each point in a sentence, predict the most probable next word given the preceding context. For example, if the first word of a sentence is "The", then a reasonable language model might assign high probabilities to nouns (e.g. "dog" or "boy") and low probabilities to verbs (e.g. "swims" or "sings") as the next word.

How can we evaluate whether a language model has successfully "learned" English grammar, aside from using our intuition? One thing we can do is test the language model on some basic grammatical phenomena, such as **subject-verb agreement.**

### 4.1 Subject-verb agreement

In English, every well-formed sentence has a **subject** and a **verb**. For example, in the sentence 

> (1) The dog sniffs a bone.

*dog* is the subject, and *sniffs* is the verb. The subject and verb must **agree** in **number** -- that is, a singular subject must be paired with a singular verb, and a plural subject must be paired with a plural verb. Returning to our example, this simply means that

> (1) The dog sniffs a bone.
>
> (2) The dogs sniff a bone.
  
are both grammatically well-formed, but 

> (3) \*The dog sniff a bone.
>
> (4) \*The dogs sniffs a bone.
  
are not. (That's what the asterisk means!)

### 4.2 Evaluation metric

How does this relate to LSTM language models? The key assumption is that a language model that has successfully "learned" English should assign **higher probability** to grammatically well-formed sentences ("grammatical"), and **lower probability** to grammatically ill-formed sentences ("ungrammatical").

This gives us a metric for evaluating our language model. Recall that language models assign probabilities at the level of each word. So a successful model should assign

\begin{align*}
P(\text{singular verb }|\text{ singular subject}) &> P(\text{plural verb }|\text{ singular subject}) \\
P(\text{plural verb }|\text{ plural subject}) &> P(\text{singular verb }|\text{ plural subject}).
\end{align*}

Returning to our example, this means that we would expect

\begin{align*}
P(\text{sniffs }|\text{ The dog}) &> P(\text{sniff }|\text{ The dog}) \\
P(\text{sniff }|\text{ The dogs}) &> P(\text{sniffs }|\text{ The dogs}).
\end{align*}

Finally, for reasons beyond the scope of this tutorial, psycholinguists are typically interested in the **negative log probability** of words, as [it is better correlated with human behavioral measures](https://www.mit.edu/~rplevy/papers/smith-levy-2013-cognition.pdf). Since this quantity is known as **surprisal**, let's denote it by $S$. This means that we will equivalently be looking for the pattern 

\begin{align*}
S(\text{sniffs }|\text{ The dog}) &< S(\text{sniff }|\text{ The dog}) \\
S(\text{sniff }|\text{ The dogs}) &< S(\text{sniffs }|\text{ The dogs}).
\end{align*}

So, does this actually hold in practice? Let's take a look below!

### 4.3 Evaluation

First, let's import some helper functions from `utils.py` that we'll need for evaluation.

In [3]:
import utils

Next, we need to download the parameters for the pre-trained model. This language model has been trained on an Americanized version of the [British National Corpus (BNC)](https://www.english-corpora.org/bnc/). You can **download the model checkpoint** [here](https://www.dropbox.com/s/er9exdbwun4rex9/model_bnc.pt?dl=1). Clicking on the link should download a file called `model_bnc.pt`, which you should move to the `materials` folder.

Once you've done that, let's set some parameters and paths to important information: the model checkpoint, input sentences, and dictionaries containing vocabulary information.

In [4]:
params = {
    'checkpoint': 'materials/model_bnc.pt', # path to model checkpoint
    'eval_data': 'materials/examples.txt', # path to eval data
    'dicts': 'materials/lstm_bnc_dicts.json', # path to dictionaries
    'temperature': 1.0, # higher temperature will increase diversity
    'seed': 1111 # random seed for reproducibility
}

Here, we'll initialize some settings, e.g. random seed and device (CPU or GPU).

In [5]:
torch.manual_seed(params['seed'])
device = torch.device("cpu")
assert(params['temperature'] >= 1e-3)

Load and initialize the model from the specified path.

In [7]:
with open(params['checkpoint'], 'rb') as f:
    model = torch.load(f, map_location=lambda storage, loc: storage)
    model.cpu()
model.eval()

RNNModel(
  (drop): Dropout(p=0.5)
  (encoder): Embedding(68344, 400)
  (rnn): LSTM(400, 400, num_layers=2, dropout=0.5)
  (decoder): Linear(in_features=400, out_features=68344, bias=True)
)

Load the sentences and dictionaries from the specified paths. Note that the sentence file `params['eval_data']` is expected to have one sentence on each line, with tokens separated by spaces. (For the simple sentences that we'll be working with, just think of separating each word with a space, as well as the final period.)

In [8]:
# load eval sentences
with open(params['eval_data'], 'r') as f:
    lines = f.readlines()
sents = [line.strip().split() for line in lines]

# load the dictionaries, word2idx and idx2word, of the preprocessed corpus used by the model
corpus_dict = utils.Dictionary()
corpus_dict.load(params['dicts'])
ntokens = len(corpus_dict)

Finally, we're ready to perform the evaluation. Each word and its negative log probability will be printed on a new line, separated by a tab (`\t`).

In [9]:
input = torch.randint(ntokens, (1, 1), dtype=torch.long).to(device)

with torch.no_grad():
    for sent in sents:
        # print the sentence
        print('SENTENCE:', ' '.join(sent))
        
        # initialize the hidden layer
        hidden = model.init_hidden(1)

        # map the first word of the sentence to its index in the model vocabulary
        input = torch.tensor([[corpus_dict.word2idx[sent[0]]]],dtype=torch.long).to(device)
        print(sent[0]+'\t'+'0.0')

        for i, w in enumerate(sent[1:]):
            output, hidden = model(input, hidden)
            word_weights = torch.Tensor.numpy(output.squeeze().div(params['temperature']).exp().cpu())
            total_weight = np.sum(word_weights)
            word_idx = corpus_dict.word2idx[w]
            word_surprisal = -np.log(word_weights[word_idx]/total_weight)
            print(w+'\t'+str(word_surprisal))
            input.fill_(word_idx)
        print()

SENTENCE: The dog sniffs a bone .
The	0.0
dog	9.9091015
sniffs	12.022201
a	3.2705405
bone	7.8615108
.	2.0750206

SENTENCE: The dogs sniff a bone .
The	0.0
dogs	11.95605
sniff	12.105198
a	3.5777009
bone	8.10669
.	2.096223

SENTENCE: The dog sniff a bone .
The	0.0
dog	9.9091015
sniff	12.718703
a	3.542921
bone	7.916548
.	2.004443

SENTENCE: The dogs sniffs a bone .
The	0.0
dogs	11.95605
sniffs	13.675603
a	3.714163
bone	8.178829
.	2.148306



We see 

\begin{align*}
S(\text{sniffs }|\text{ The dog}) = 12.02 &< S(\text{sniff }|\text{ The dog}) = 12.72 \;\; \checkmark \\
S(\text{sniff }|\text{ The dogs}) = 12.11 &< S(\text{sniffs }|\text{ The dogs}) = 13.68 \;\; \checkmark
\end{align*}

as expected. This suggests that the language model has indeed learned something about subject-verb number agreement!

### 4.4 Exploration

Now, it's your turn to play around with the model by designing your own test sentences. You can edit `materials/examples.txt` directly and re-run all the cells after loading the eval data, or make your own file following the format of `materials/examples.txt` and update `params['eval_data']` accordingly. 

Do the results surprise you? What other grammatical phenomena would be good for assessing language models?